[View in Colaboratory](https://colab.research.google.com/github/etoi2pi/IoT-anomaly-detection/blob/master/IoT_devices.ipynb)

In [28]:
import numpy as np
import tensorflow as tf
import timeit
from collections import Counter
import datetime as dt
import pandas as pd
import re
from timeit import default_timer as timer # time the code

# housekeeping for Google
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#make sure we are at lastest version of pip
!pip install --upgrade pip
# get latest package versions
!pip install -U gspread
import gspread
!pip install -U xgboost
import xgboost
!pip install -U Keras


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')


print('last run: ',dt.datetime.now())

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (18.0)
Requirement already up-to-date: gspread in /usr/local/lib/python3.6/dist-packages (3.0.1)
Requirement already up-to-date: xgboost in /usr/local/lib/python3.6/dist-packages (0.72.1)
Requirement already up-to-date: Keras in /usr/local/lib/python3.6/dist-packages (2.2.2)
last run:  2018-08-01 10:14:46.248199


In [29]:
#!pip install -U -q PyDrive
#import os
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


file_id_train = '14GmlKkyKBHoibTL7Oynq8w9Eq4BXTAqo' # iot_train.csv
file_id_valid = '1XtX67OqsKeQmOFFcUxrY7kLyFGjBgANs' # iot_validation.csv
file_id_test = '1fUt8rm_jYkCQEwtruJTe4068KcXgccDp' # iot_test.csv
downloaded_valid = drive.CreateFile({'id': file_id_valid})
downloaded_train = drive.CreateFile({'id': file_id_train})
downloaded_test = drive.CreateFile({'id': file_id_test})
# allows you to temporarily load your file in the notebook VM

# assume the file is called file.csv and it's located at the root of your drive
downloaded_valid.GetContentFile('iot_validation.csv')
downloaded_test.GetContentFile('iot_test.csv')
downloaded_train.GetContentFile('iot_train.csv')

# time the download
start = timer()
pdIoTValid = pd.read_csv('iot_validation.csv')
pdIoTTest = pd.read_csv('iot_test.csv')
pdIoTTrain = pd.read_csv('iot_train.csv')
end = timer()
print('file load time:',end - start, 'seconds\n')

# check on our files
print('train:', pdIoTTrain.shape, 'test:', pdIoTTest.shape, 'valid:', pdIoTValid.shape)
print('last run: ',dt.datetime.now())

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,242,243,244,245,246,247,248,249) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


file load time: 18.32718499100156
train: (399543, 298) test: (1000, 302) valid: (1000, 302)
last run:  2018-08-01 10:15:25.053393


In [0]:
# 14GmlKkyKBHoibTL7Oynq8w9Eq4BXTAqo - iot_train.csv
# 1XtX67OqsKeQmOFFcUxrY7kLyFGjBgANs - iot_validation.csv
# 1fUt8rm_jYkCQEwtruJTe4068KcXgccDp - iot_test.csv

!ls -l
# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk' in parents"}).GetList()

print(file_list)
"""
#for f in file_list:
if file_list[0] is not None:
  f = file_list[0]
  print(f, 'is filename \n')
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


with open(fname, 'r') as f:
  print(f.read())
  """


In [0]:


gc = gspread.authorize(GoogleCredentials.get_application_default()) #.authorize(gauth.credentials)
try:
  validation_fn = 'challengeToFill.csv'#'predictions_out_final.csv'#'iot_validation.csv'
  wb_iot_validation = gc.open(validation_fn)
except gspread.SpreadsheetNotFound:
  print(validation_fn,'not Found')
#dir(gc)

In [0]:
usercommands = []
for i in range(39):
  fname = 'datalab/User' + str(i)
  f = open(fname,'r')
  rawstr = f.read()
  #print(rawstr[0:10])
  # originally found 15094 words, but that's because there are 47 occurences of 
  # . a dash '-' in emacs-20 command, so it separated it into 2 words (2*47=94)
  # . also need to get rid of the \n
  cleanstr = re.sub('[-]','',rawstr) 
  cleanstr = re.sub('[\\n]',' ',cleanstr)
  #print(len(cleanstr))
  usercommands.append(cleanstr)
  #print(len(message),type(message))
  f.close()
print('\n---\n',len(usercommands),usercommands[0])


In [14]:



#('My Drive/AI cyber attacks/Final project/IoT device type identification/hackathon_IoT_training_set_based_on_01mar2017.csv')
ws_test = wb_iot_test.sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
iot_trainset = pd.DataFrame.from_records(rows)


SpreadsheetNotFound: ignored

In [0]:
# Keras Models
from keras.models import Sequential

model = Sequential()